In [1]:
from flask import Flask, render_template_string, request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
import time
import threading

# HTML Template
HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>Indian Court Case Lookup</title>
</head>
<body>
    <h2>Search Indian Court Case</h2>
    <form method="POST">
        <label for="case_type">Case Type:</label>
        <input type="text" name="case_type" required><br><br>
        <label for="case_number">Case Number:</label>
        <input type="text" name="case_number" required><br><br>
        <input type="submit" value="Search">
    </form>
    {% if result %}
        <h3>Result:</h3>
        <pre>{{ result }}</pre>
    {% endif %}
</body>
</html>
"""

# Scraper function
def get_case_details(case_type, case_number):
    try:
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)

        driver.get("https://services.ecourts.gov.in/ecourtindia_v6/")
        time.sleep(3)

        # Select State
        Select(driver.find_element(By.ID, "sess_state_code")).select_by_visible_text("Haryana")
        time.sleep(2)

        # Select District
        Select(driver.find_element(By.ID, "sess_dist_code")).select_by_visible_text("Faridabad")
        time.sleep(2)

        # Click on "Case Status"
        driver.find_element(By.LINK_TEXT, "Case Status").click()
        time.sleep(3)

        # Fill in form
        Select(driver.find_element(By.ID, "ctype")).select_by_visible_text(case_type)
        driver.find_element(By.ID, "cnumber").send_keys(case_number)
        Select(driver.find_element(By.ID, "cyear")).select_by_value("2023")

        driver.find_element(By.ID, "submitbtn").click()
        time.sleep(5)

        # Try to extract case details
        try:
            case_info = driver.find_element(By.CLASS_NAME, "case_details_table").text
        except:
            case_info = "❌ No case data found or CAPTCHA required."

        driver.quit()
        return case_info
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Flask App
app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def index():
    result = None
    if request.method == "POST":
        case_type = request.form["case_type"]
        case_number = request.form["case_number"]
        result = get_case_details(case_type, case_number)
    return render_template_string(HTML, result=result)

# Run Flask in a thread
def run_app():
    app.run(port=5000, debug=False, use_reloader=False)

threading.Thread(target=run_app).start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Aug/2025 14:59:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2025 14:59:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Aug/2025 15:00:27] "POST / HTTP/1.1" 200 -
